In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import datetime

Using TensorFlow backend.


In [6]:
np.random.seed(7)

In [7]:
def normalize(column):
    new_column = []
    min_val = min(column)
    max_val = max(column)
    range_val = max_val-min_val
    for entry in column:
        new_column.append((entry-min_val)/range_val)
    return new_column

In [29]:
train_labels = pd.read_csv('./input/dengue_labels_train.csv', engine='python')
total_test_features = pd.read_csv('./input/dengue_features_test.csv')
train_labels = np.ravel(train_labels.drop(train_labels.columns[[0, 1, 2]], axis=1).values.astype('float32'))
# test_labels = pd.read_csv('./input/dengue_labelste.csv')
# sj_final_test = test_labels[:260]
# iq_final_test = test_labels[261:]
sj_labels = total_labels[935:]
iq_labels = total_labels[:936]

In [41]:
dataset = sj_labels
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit_transform(dataset)
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
sj_train, sj_test = dataset[0:train_size], dataset[train_size:len(dataset)]

dataset = iq_labels
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit_transform(dataset)
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
iq_train, iq_test = dataset[0:train_size], dataset[train_size:len(dataset)]

/Users/Simran/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/Simran/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/Simran/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if yo

In [42]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [44]:
look_back = 1
sj_trainX, sj_trainY = create_dataset(sj_train, look_back)
sj_testX, sj_testY = create_dataset(sj_test, look_back)
iq_trainX, iq_trainY = create_dataset(iq_train, look_back)
iq_testX, iq_testY = create_dataset(iq_test, look_back)
# sj_x, sj_y = create_dataset(sj_final_test, look_back)
# iq_x, iq_y = create_dataset(iq_final_test, look_back)

In [47]:
# reshape input to be [samples, time steps, features]
sj_trainX = np.reshape(sj_trainX, (sj_trainX.shape[0], 1, sj_trainX.shape[1]))
sj_testX = np.reshape(sj_testX, (sj_testX.shape[0], 1, sj_testX.shape[1]))

# reshape input to be [samples, time steps, features]
iq_trainX = np.reshape(iq_trainX, (iq_trainX.shape[0], 1, iq_trainX.shape[1]))
iq_testX = np.reshape(iq_testX, (iq_testX.shape[0], 1, iq_testX.shape[1]))

# sj_x = np.reshape(sj_x, (sj_x.shape[0], 1, sj_x.shape[1]))
# iq_x = np.reshape(iq_x, (iq_x.shape[0], 1, iq_x.shape[1]))

In [48]:
print(sj_trainY)

[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.
    0.    0.    0.    1.    1.    0.    0.    0.    0.    0.    0.    0.
    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    1.    1.
    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    1.    1.    1.    2.    4.    1.    4.   11.   16.   23.   12.   14.
   18.    8.    7.   10.    7.   10.    5.   11.    8.   18.   13.    9.
   22.   10.    5.   13.    2.   11.   11.    3.    7.    7.    4.    5.
    6.    7.    7.    4.    9.   17.    8.   22.   18.   21.   16.   31.
   25.   28.   26.   18.   27.   11.   38.   29.   21.   11.   10.    5.
    6.    2.    1.    2.    2.    3.    5.    1.    4.    2.    4.    0.
    0.    0.    0.    1.    1.    1.    1.    1.    2.    3.    4.    6.
    2.    2.    5.    1.    1.    0.    0.    0.   

In [52]:
sj_trainY

array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    1.,    0.,    0.,    0.,    0.,    1.,    1.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,
          0.,    1.,    1.,    0.,    0.,    1.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          1.,    1.,    1.,    2.,    4.,    1.,    4.,   11.,   16.,
         23.,   12.,   14.,   18.,    8.,    7.,   10.,    7.,   10.,
          5.,   11.,    8.,   18.,   13.,    9.,   22.,   10.,    5.,
         13.,    2.,   11.,   11.,    3.,    7.,    7.,    4.,    5.,
          6.,    7.,    7.,    4.,    9.,   17.,    8.,   22.,   18.,
         21.,   16.,   31.,   25.,   28.,   26.,   18.,   27.,   11.,
         38.,   29.,

In [49]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(sj_trainX, sj_trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
5s - loss: 140.8208
Epoch 2/100
2s - loss: 130.3388
Epoch 3/100
2s - loss: 123.7421
Epoch 4/100
2s - loss: 118.9274
Epoch 5/100
3s - loss: 114.8861
Epoch 6/100
2s - loss: 111.4296
Epoch 7/100
3s - loss: 108.3594
Epoch 8/100
2s - loss: 105.6781
Epoch 9/100
2s - loss: 103.3232
Epoch 10/100
2s - loss: 101.2203
Epoch 11/100
2s - loss: 99.3314
Epoch 12/100
2s - loss: 97.6510
Epoch 13/100
2s - loss: 96.1166
Epoch 14/100
2s - loss: 94.6689
Epoch 15/100
2s - loss: 93.3444
Epoch 16/100
3s - loss: 92.1244
Epoch 17/100
3s - loss: 90.9757
Epoch 18/100
2s - loss: 89.9037
Epoch 19/100
3s - loss: 88.9125
Epoch 20/100
2s - loss: 87.9666
Epoch 21/100
2s - loss: 87.0908
Epoch 22/100
2s - loss: 86.2434
Epoch 23/100
2s - loss: 85.4608
Epoch 24/100
2s - loss: 84.6847
Epoch 25/100
2s - loss: 83.9594
Epoch 26/100
2s - loss: 83.2924
Epoch 27/100
2s - loss: 82.5993
Epoch 28/100
3s - loss: 81.9736
Epoch 29/100
2s - loss: 81.2862
Epoch 30/100
2s - loss: 80.7048
Epoch 31/100
2s - loss: 80.1303
Epoch 3

In [50]:
# make predictions
trainPredict = model.predict(sj_trainX)
testPredict = model.predict(sj_testX)
sj_predictions = [int(i) for i in model.predict(sj_x)]
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([sj_trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([sj_testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

NameError: name 'sj_x' is not defined

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(iq_trainX, iq_trainY, epochs=100, batch_size=1, verbose=2)

In [ ]:
# make predictions
trainPredict = model.predict(iq_trainX)
testPredict = model.predict(iq_testX)
iq_predictions = [int(i) for i in model.predict(iq_x)]
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([iq_trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([iq_testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
print(iq_trainX)

In [ ]:
print(iq_testX)

In [339]:
print((len(iq_x.ravel())))

518


In [335]:
print(len(iq_predictions))

518


In [334]:
submission = pd.read_csv('./input/submission_format.csv',
                         index_col=[0, 1, 2])

submission.total_cases = np.concatenate([sj_predictions, iq_predictions])
submission.to_csv('./input/lstm_submission.csv')

ValueError: Length of values does not match length of index